# EXPERIMENT 3

### SHIVAM SINGHAL 2K18/CO/340

**AIM:** Write a program to exhibit the working of the decision tree based ID3 algorithm. With the help of appropriate data set build the decision tree and classify a new sample.

In [4]:
import pandas as pd
import numpy as np 

In [5]:
df = pd.read_csv('main-bank.csv')

In [6]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 4.1+ MB


In [8]:
df.housing.unique()

array(['yes', 'no'], dtype=object)

In [9]:
df = df.drop(['age' , 'balance' , 'day' , 'month' ,'contact', 'duration' , 'campaign' , 'pdays' , 'previous' ], 1)

In [10]:
# Calculating Entropy 

In [11]:
def calculate_entropy(df):
    entropy_node = 0  
    values = df.y.unique()  
    for value in values:
        fraction = df.y.value_counts()[value]/len(df.y)  
        entropy_node += -fraction*np.log2(fraction)
    return entropy_node

In [12]:
def info_gain(df , col):
    gain = calculate_entropy(df)
    atributes = df[col].unique()
    count = df[col].count()
    for atr in atributes: 
        ent = calculate_entropy(df[df[col] == atr])
        cnt = df[df[col] == atr][col].count()
        ent *= cnt/count
        gain -= ent
    return gain

In [13]:
def find_winner(df):
    Info_Gain = []
    for key in df.keys()[:-1]:
        Info_Gain.append(info_gain(df , key))
    return df.keys()[:-1][np.argmax(Info_Gain)]

In [14]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1] 
    node = find_winner(df)
    attValue = np.unique(df[node])
    
    if tree is None:                    
        tree={}
        tree[node] = {}

    for value in attValue:
        
        subtable = df[df[node] == value].drop(node , 1)
        
        total_result = subtable.y.count()
        
        arr = subtable.y.unique()
        
        cnt2 = len(subtable.keys())
        
        if len(arr) == 1:
            tree[node][value] = arr[0]
        elif cnt2 == 1:
            yes_count = subtable[subtable['y'] == 'yes']['y'].count()
            no_count = subtable[subtable['y'] == 'no']['y'].count() 
            if(yes_count > no_count) :
                tree[node][value] = 'yes'
            else :
                tree[node][value] = 'no'
        else:
            tree[node][value] = buildTree(subtable)
                   
    return tree

In [15]:
df.head()

,job,marital,education,default,housing,loan,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,no
1,technician,single,secondary,no,yes,no,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,no
4,unknown,single,unknown,no,no,no,unknown,no


In [16]:
from sklearn.utils import shuffle
df = shuffle(df, random_state = 1)

In [17]:
df.head()
print(len(df))

45211


In [18]:
#partition into training test data
train_len = len(df)*6//10
train_df = df.iloc[:train_len]
test_df = df.iloc[train_len:]
print(len(train_df))
print(len(test_df))

27126
18085


In [19]:
test_df.head()

,job,marital,education,default,housing,loan,poutcome,y
1447,management,married,tertiary,no,yes,no,unknown,no
25576,retired,married,primary,no,no,no,unknown,no
5622,management,divorced,tertiary,no,no,no,unknown,no
9691,blue-collar,married,unknown,no,no,no,unknown,yes
3401,services,divorced,primary,no,yes,no,unknown,no


In [20]:
decision_tree = buildTree(train_df)

In [21]:
# This function will gives the number of correct result from the test dataset
def check(node, tree , test_df ):
    TP = TN = 0
    if len(test_df) == 0 :
        return 0,0
    for child,value in tree.items():
        if value == 'yes' :
            TP += len(test_df[test_df[node]== child][test_df[test_df[node]== child]['y'] == value])
        elif value == 'no' :
            TN += len(test_df[test_df[node]== child][test_df[test_df[node]== child]['y'] == value])
        else :
            next_node = list(value.keys())[0]
            P,N = check(next_node , tree[child][next_node], test_df[test_df[node]== child])
            TP += P
            TN += N
    return TP,TN

In [22]:
next_node = list(decision_tree.keys())[0]
TP,TN = check(next_node, decision_tree[next_node] , test_df)
total_outcome = len(test_df)

In [23]:
FP = len(test_df[test_df['y']=='yes']) - TP
FN = len(test_df[test_df['y']=='no']) - TN

In [24]:
accuracy= (TP+TN) / (TP+FP+TN+FN) * 100
print(accuracy)

88.26098977052807


In [25]:
precision = TP / (TP+FP)
print(precision*100)

16.08325449385052


In [26]:
recall = TP / (TP+FN)
print(recall*100)

49.34687953555878


In [27]:
f1_score = 2*(recall * precision) / (recall + precision)
print(f1_score*100)

24.25972172672137


In [28]:
specificity = TN / (TN + FP)
print(specificity*100)

89.80225339158427
